In [49]:
staff = {"design": 20,
"uiux": 5,
"frontend": 10,
"data": 15,
"backend": 5,
"account": 20,
"maint": 1,
"exec": 10,
"ceo": 1,
"dog": 1
}

In [50]:
conditions = { 
            0: {
                "max_num": 10,
                "max_dist": 5,
                "people": staff["design"]
                "description": "Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design."
                "source": "Crunchbase",
                "categories": "{$or: [{tag_list:{$regex : "design"}}, {description:{$regex : "design"}}, {name:{$regex : "design"}}]}""
            }
    
            1: {
                "max_num": 5,
                "max_dist": 3,
                "people": round(sum(staff.values())*0.3),
                "description": "30% of the company staff have at least 1 child.",
                "source": "Foursquare",
                "categories": [12055, 12056, 12057, 12058, 12059, 12060, 12061, 12062],
                "query": None
             },

              2: {
                  "max_num": 20,
                  "max_dist": 20,
                  "people": staff["frontend"] + staff["backend"],
                  "description": "Developers like to be near successful tech startups that have raised at least 1 Million dollars.",
                  "source": "Crunchbase",
                  "categories": "total_money_raised"
                  "query": "More than 1,000,000"
              },
              3: {
                  "max_num": 1,
                  "max_dist": 2,
                  "people": staff["exec"],
                  "description": "Executives like Starbucks A LOT. Ensure there's a starbucks not too far."
                  "source": "Foursquare",
                  "category":[13032]
                  "query": "Starbucks"
                  
              },
              4: {
                  "max_num": 1,
                  "max_dist": 50,
                  "people": staff["account"],
                  "description": "Account managers need to travel a lot."
                  "source": "Foursquare",
                  "category":[19031, 19040]
                  "query": None
              },
              5: {"max_num": 30,
                  "max_dist": 3,
                  "people": sum(staff.values()),
                  "description": "Everyone in the company is between 25 and 40, give them some place to go party."
                  "source": "Foursquare",
                  "category":[13003, 10032, ]
                  "query": None
              },
              6: {
                  "max_num": 2,
                  "max_dist": 2,
                  "people": staff["ceo"],
                  "description":"The CEO is vegan.",
                  "source": "Foursquare",
                  "category":[13377]
                  "query": None
              },
              7: {
                  "max_num": 1,
                  "max_dist": 10,
                  "people": staff["maint"],
                  "description": "If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km."
                  "source": "Foursquare",
                  "category":[18008]
                  "query": None
              },
              8: {
                  "max_num": 1,
                  "max_dist": 4,
                  "people": staff["dog"],
                  "description": "The office dog Dobby needs a hairdresser every month. Ensure there's one not too far away."
                  "source": "Foursquare",
                  "category":[11134]
                  "query": None
              },
}

In [51]:
def score_calculator(condition, num, min_dist):
    if num > 0:
        distance = 1 / min_dist
        if num > conditions[condition]['max_num']:
            number = 1
        else:
            number = num / conditions[condition]['max_num']
        score = (number + distance)*conditions[condition]["people"]
    else:
        score = 0
    return score

In [52]:
import pandas as pd
pd.DataFrame(conditions)

,1,2,3,4,5,6,7,8
max_num,5,20,1,1,30,2,1,1
max_dist,3,20,2,50,3,2,10,4
people,26,15,10,20,88,1,1,1
description,30% of the company staff have at least 1 child.,Developers like to be near successful tech sta...,Executives like Starbucks A LOT. Ensure there'...,Account managers need to travel a lot.,"Everyone in the company is between 25 and 40, ...",The CEO is vegan.,"If you want to make the maintenance guy happy,...",The office dog—Dobby needs a hairdresser every...


## cond 2

In [108]:
from pymongo import MongoClient
import pandas as pd
import time
client = MongoClient("localhost:27017")
db=client['Ironhack']
c =db.get_collection('lab')


In [112]:
query = {"total_money_raised":{"$exists":"true"}}
projection = {"name":1, "total_money_raised":1, "offices":1, "_id":0}
companies = list(c.find(query, projection))

In [113]:
def clean_money (string):
    '''
    Function to convert to a number de string located in the field total_money_raised of the Crunchbase database
    '''
    money = ""
     # Currency convertion value 29/10/2022
    if "C$" in string:
        multi = 0.73
        string = string.replace("C$", "")
    elif "$" in string:
        multi = 1
        string = string.replace("$", "")
    elif "€" in string:
        multi = 1
        string = string.replace("€", "")
    elif "£" in string:
        multi = 1.16
        string = string.replace("£", "")
    elif "¥" in string:    
        multi = 0.0068 #All companies that contain ¥ are located in Japan
        string = string.replace("¥", "")
    elif "kr" in string:
        multi = 0.091
        string = string.replace("kr", "")
    # Transforming string to a number
    for i in string:
            if i.isdigit() or i== ".":
                money = money + i
            elif i == 'K' or i == 'k':
                multi = 1000
            elif i == 'M' or i == 'm':
                multi = 1000000
            elif i == 'B' or i == 'b':
                multi = 1000000000
            else:
                print(string)
                return (string)
    return float(money)*multi
        

In [114]:
companies_df = pd.DataFrame(companies)
companies_df.dropna(inplace=True)

In [115]:
companies_df["total_money_raised"] = companies_df["total_money_raised"].apply(clean_money)
companies_df = companies_df[companies_df["total_money_raised"] > 1000000]

In [116]:
companies_df.shape

(3922, 3)

In [118]:
offices_df = []
for i, v in companies_df.iterrows():
    for e in v["offices"]:
        office = {}
        office['name'] = v['name']
        office['country_code'] = e['country_code']
        office['city'] = e['city']
        office['latitude'] = e['latitude']
        office['longitude'] = e['longitude']
        offices_df.append(office)
    

In [119]:
offices_df = pd.DataFrame(offices_df)

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,Digg,USA,San Francisco,37.764726,-122.394523
3,Facebook,USA,Menlo Park,37.416050,-122.151801
4,Facebook,IRL,Dublin,53.344104,-6.267494
...,...,...,...,...,...
5071,Celestial Semiconductor,USA,San Jose,37.386417,-121.929170
5072,Purfresh,USA,Menlo Park,37.477008,-122.151599
5073,InSound Medical,USA,Newark,37.512038,-121.998513
5074,Getyoo,BEL,Brussels,50.839970,4.346472


In [127]:
offices_df=offices_df[offices_df['latitude'].notna()]

In [128]:
offices_df

,name,country_code,city,latitude,longitude
0,Wetpaint,USA,Seattle,47.603122,-122.333253
1,Wetpaint,USA,New York,40.723731,-73.996431
2,Digg,USA,San Francisco,37.764726,-122.394523
3,Facebook,USA,Menlo Park,37.416050,-122.151801
4,Facebook,IRL,Dublin,53.344104,-6.267494
...,...,...,...,...,...
5069,Cantimer,USA,Menlo Park,37.484616,-122.181753
5071,Celestial Semiconductor,USA,San Jose,37.386417,-121.929170
5072,Purfresh,USA,Menlo Park,37.477008,-122.151599
5073,InSound Medical,USA,Newark,37.512038,-121.998513
